# Generate for Resnet


In [1]:
from torch.onnx import export
import torch
import torchvision.models as models

model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.eval()
dummy_input = torch.randn(1, 3, 224, 224)  # Example input with batch size 1

export(model, dummy_input, "../onnx_models/resnet50.onnx", opset_version=12,
       input_names=['input'], output_names=['output'],
       dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})


# Generate for BERT

In [1]:
from transformers import BertModel
from torch.onnx import export
import torch

# Load the pretrained BERT model and tokenizer
model = BertModel.from_pretrained("bert-base-uncased")
model.eval()

# Define static inputs
sequence_length = 128
batch_size = 1
input_ids = torch.randint(0, 30522, (batch_size, sequence_length), dtype=torch.int64)
attention_mask = torch.ones((batch_size, sequence_length), dtype=torch.int64)

# Export the model to ONNX with fixed dimensions
export(
    model,
    args=(input_ids, attention_mask),
    f="../onnx_models/bert-base.onnx",
    opset_version=19,
    input_names=["input_ids", "attention_mask"],
    output_names=["last_hidden_state", "pooler_output"]
)

print("Exported BERT model with static shapes to ONNX!")

2025-02-04 10:11:42.989807: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-04 10:11:44.054617: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738644104.427807 4086798 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738644104.530204 4086798 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-04 10:11:45.336858: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Exported BERT model with static shapes to ONNX!


# Generate for LSTM


In [ ]:
import math
import torch
import torch.nn as nn
from torch.onnx import export

# Define a simple LSTM model
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=num_layers,
            dropout=dropout_rate,
            batch_first=True,
        )
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, vocab_size)

        if tie_weights:
            assert embedding_dim == hidden_dim, "cannot tie, check dims"
            self.embedding.weight = self.fc.weight
        self.init_weights()

    def forward(self, src, hidden):
        embedding = self.dropout(self.embedding(src))
        output, hidden = self.lstm(embedding, hidden)
        output = self.dropout(output)
        prediction = self.fc(output)
        return prediction, hidden

    def init_weights(self):
        init_range_emb = 0.1
        init_range_other = 1 / math.sqrt(self.hidden_dim)
        self.embedding.weight.data.uniform_(-init_range_emb, init_range_emb)
        self.fc.weight.data.uniform_(-init_range_other, init_range_other)
        self.fc.bias.data.zero_()
        for i in range(self.num_layers):
            self.lstm.all_weights[i][0] = torch.FloatTensor(
                self.embedding_dim, self.hidden_dim
            ).uniform_(-init_range_other, init_range_other)
            self.lstm.all_weights[i][1] = torch.FloatTensor(
                self.hidden_dim, self.hidden_dim
            ).uniform_(-init_range_other, init_range_other)

    def init_hidden(self, batch_size, device):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        return hidden, cell

    def detach_hidden(self, hidden):
        hidden, cell = hidden
        hidden = hidden.detach()
        cell = cell.detach()
        return hidden, cell


# Parameters
vocab_size = 29423
embedding_dim = 256  # Match hidden_dim for tied weights
hidden_dim = 256
num_layers = 2
dropout_rate = 0.2
tie_weights = True  # Tied weights now valid
num_epochs = 1
batch_size = 1
seq_length = 32

# Initialize the model
lstm_model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights)
lstm_model.eval()  # Set to evaluation mode

# Example input for the LSTM model
inputs = torch.randint(0, vocab_size, (batch_size, seq_length), dtype=torch.long)
hidden_ = (torch.zeros(num_layers, batch_size, hidden_dim),
           torch.zeros(num_layers, batch_size, hidden_dim))

# Export the model to ONNX format
onnx_file_path = "../onnx_models/lstm.onnx"
export(
    lstm_model,
    args=(inputs, hidden_),  # Model input with optional hidden state
    f=onnx_file_path,
    opset_version=19,  # Compatible ONNX opset for LSTM
    input_names=["input", "hidden"],  # Input tensor name
    output_names=["output", "hidden_out"],  # Output tensor names
    dynamic_axes={
        "input": {0: "batch_size", 1: "sequence_length"},  # Dynamic axes
        "hidden": {1: "batch_size"},
    },
)

print(f"LSTM model exported to {onnx_file_path}!")


# Generate for YOLO

In [ ]:
from ultralytics import YOLO
import os

# Specify model weights and export path
model_weights = 'yolov8n.pt'  # Path to YOLOv8-n weights
export_path = '../onnx_models/'  # Desired export path

# Load the YOLOv8 model
model = YOLO(model_weights)

# Export the model to ONNX format
model.export(format='onnx', imgsz=640, dynamic=False)
!mv yolov8n.onnx export_path

print(f"Model exported to ONNX format at: {export_path}")

# Generate for MobileNet

In [5]:
from torch.onnx import export
import torch
import torchvision.models as models

model = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.DEFAULT)
model.eval()
dummy_input = torch.randn(1, 3, 224, 224)  # Example input with batch size 1

export(model, dummy_input, "../onnx_models/mobnet.onnx", opset_version=19,
       input_names=['input'], output_names=['output'],
       dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})


In [ ]:
from torch.onnx import export
import torch
import torchvision.models as models

model = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.DEFAULT)
model.eval()
layer = model.features[16][0]

dummy_input = torch.randn(1, 160, 7, 7)  # Example input with batch size 1

export(layer, dummy_input, "../onnx_models/mobnet_pw_conv.onnx", opset_version=19,
       input_names=['input'], output_names=['output'],
       dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})

In [4]:
from torch.onnx import export
import torch
import torchvision.models as models

model = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.DEFAULT)
model.eval()
layer = model.features[4]
conv1 = layer.block[0][0]

dummy_input = torch.randn(1, 24, 56, 56)  # Example input with batch size 1

export(conv1, dummy_input, "../onnx_models/mobnet_bottleneck_conv1.onnx", opset_version=19,
       input_names=['input'], output_names=['output'],
       dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})

/home/saisamarth/miniconda3/lib/python3.11/site-packages/torch/onnx/utils.py:1531: OnnxExporterWarning: Exporting to ONNX opset version 19 is not supported. by 'torch.onnx.export()'. The highest opset version supported is 17. To use a newer opset version, consider 'torch.onnx.dynamo_export()'. Note that dynamo_export() is in preview. Please report errors with dynamo_export() as Github issues to https://github.com/pytorch/pytorch/issues.
  warnings.warn(


In [6]:
from torch.onnx import export
import torch
import torchvision.models as models

conv1 = torch.nn.Conv2d(160, 960, (1, 1), (1, 1), bias=False).eval()

dummy_input = torch.randn(1, 160, 56, 56)  # Example input with batch size 1

export(conv1, dummy_input, "../onnx_models/mobnet_bottleneck_conv2.onnx", opset_version=12,
       input_names=['input'], output_names=['output'],
       dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})

# Generate for FCN

In [ ]:
import torch
from torch.onnx import export

import torch.nn as nn

# Define a simple linear model
class SimpleLinearModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleLinearModel, self).__init__()
        self.weight = nn.Parameter(torch.randn(output_dim, input_dim))
        self.bias = nn.Parameter(torch.randn(output_dim))

    def forward(self, x):
        return torch.matmul(x, self.weight.T) + self.bias

# Parameters
input_dim = 64
output_dim = 128

# Initialize the model
linear_model = SimpleLinearModel(input_dim, output_dim)
linear_model.eval()

# Example input for the linear model
example_input = torch.randn(1, input_dim)

# Export the model to ONNX format
onnx_file_path = "../onnx_models/simple_linear_64_128.onnx"
export(
    linear_model,
    args=(example_input),
    f=onnx_file_path,
    opset_version=12,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input": {0: "batch_size"},
        "output": {0: "batch_size"}
    }
)

print(f"Linear model exported to {onnx_file_path}!")